In [3]:
import os
import random
import h5py
import json
import torch
import logging
from torch.utils.data import Dataset
import numpy as np
# from .build import DATASETS
# from utils.logger import print

def rotate_point_cloud_z(pc):
    """ Randomly rotate the point clouds to augment the dataset """
    rotation_angle = np.random.uniform() * 2 * np.pi
    cosval = np.cos(rotation_angle)
    sinval = np.sin(rotation_angle)
    rotation_matrix = np.array([[cosval, -sinval, 0],
                                [sinval, cosval, 0],
                                [0, 0, 1]])
    rotated_data = np.dot(pc, rotation_matrix)
    return rotated_data

def jitter_point_cloud(pc, sigma=0.01, clip=0.05):
    """ Randomly jitter points. jittering is per point. """
    N, C = pc.shape
    assert(clip > 0)
    jittered_data = np.clip(sigma * np.random.randn(N, C), -1 * clip, clip)
    jittered_data += pc
    return jittered_data

def random_scale_point_cloud(pc, scale_low=0.8, scale_high=1.25):
    """ Randomly scale the point cloud. Scale is per shape. """
    scale = np.random.uniform(scale_low, scale_high)
    return pc * scale


# @DATASETS.register_module()
class ShapeNetPartH5(Dataset):
    """
    Dataloader for the HDF5 version of ShapeNetPart.
    This is the standard dataset format used in PointNet/PointNet++ and subsequent works.
    """
    # 50-class mapping for ShapeNetPart
    seg_classes = {
        'Airplane': [0, 1, 2, 3], 'Bag': [4, 5], 'Cap': [6, 7], 'Car': [8, 9, 10, 11],
        'Chair': [12, 13, 14, 15], 'Earphone': [16, 17, 18], 'Guitar': [19, 20, 21],
        'Knife': [22, 23], 'Lamp': [24, 25, 26, 27], 'Laptop': [28, 29],
        'Motorbike': [30, 31, 32, 33, 34, 35], 'Mug': [36, 37], 'Pistol': [38, 39, 40],
        'Rocket': [41, 42, 43], 'Skateboard': [44, 45, 46], 'Table': [47, 48, 49]
    }
    
    # Mapping from category name to the class index (0-15)
    classes_map = {
        'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5,
        'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10,
        'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15
    }

    def __init__(self, config):
        self.root = config.DATA_PATH
        self.npoints = config.N_POINTS
        self.split = config.subset
        self.use_augmentation = (self.split == 'train')

        self.all_points = []
        self.all_seg_labels = []
        self.all_cls_labels = []

        # Find all H5 files for the given split (train/test/val)
        h5_files = [f for f in os.listdir(self.root) if f.endswith('.h5') and self.split in f]
        if not h5_files:
            raise FileNotFoundError(f"No H5 files found for split '{self.split}' in '{self.root}'")
        
        print(f"Loading H5 files for '{self.split}' split: {h5_files}")

        for h5_filename in sorted(h5_files):
            f = h5py.File(os.path.join(self.root, h5_filename), 'r')
            points = f['data'][:]
            seg_labels = f['seg'][:] 
            cls_labels = f['label'][:]
            f.close()
            
            self.all_points.append(points)
            self.all_seg_labels.append(seg_labels)
            self.all_cls_labels.append(cls_labels)

        # Concatenate data from all loaded files
        self.all_points = np.concatenate(self.all_points, axis=0)
        self.all_seg_labels = np.concatenate(self.all_seg_labels, axis=0)
        self.all_cls_labels = np.concatenate(self.all_cls_labels, axis=0).squeeze() # Squeeze to make it 1D

        print(f'The size of {self.split} data is {len(self.all_points)}')
        print(f'Number of points per sample: {self.npoints}')
        
        self.classes = self.classes_map
        
    def __len__(self):
        return len(self.all_points)

    def __getitem__(self, index):
        points = self.all_points[index][:self.npoints].copy()
        seg_labels = self.all_seg_labels[index][:self.npoints].copy()
        cls_label = self.all_cls_labels[index].copy()

        # Augmentation is applied only to the training set
        if self.use_augmentation:
            points = rotate_point_cloud_z(points)
            points = jitter_point_cloud(points)
            points = random_scale_point_cloud(points)
        
        # Normalize points
        points = self.pc_normalize(points)


        return (
            torch.from_numpy(points).float(),
            torch.from_numpy(np.array([cls_label])).long(), # Wrap in array for consistent shape
            torch.from_numpy(seg_labels).long()
        )

    @staticmethod
    def pc_normalize(pc):
        centroid = np.mean(pc, axis=0)
        pc = pc - centroid
        m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
        pc = pc / (m + 1e-9)
        return pc
    


if __name__ == "__main__":
    import sys
    import random
    # project_root = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    # sys.path.insert(0, project_root)

    class DummyConfig1:
        def __init__(self):
            # --- IMPORTANT ---
            # This path must point to the folder containing the .h5 files
            self.DATA_PATH = '/kaggle/input/shapenetpart/shapenetpart_hdf5_2048'
            self.N_POINTS = 2048
            self.subset = 'train'  # can be 'train' or 'test'

    config = DummyConfig1()
    
    print(f"--- Testing ShapeNetPart HDF5 Dataset ---")
    print(f"Loading data from: {config.DATA_PATH}")
    print(f"Subset: {config.subset}, Points per sample: {config.N_POINTS}")
    
    # 2. --- Instantiate the dataset ---
    try:
        dataset_train = ShapeNetPartH5(config)
        
    except Exception as e:
        print(f"\n[ERROR] Failed to initialize dataset. Please check the DATA_PATH in the script.")
        print(f"Details: {e}")
        exit()
    # 3. --- Get and inspect a random sample ---
    if len(dataset_train) == 0:
        print("\n[ERROR] The dataset is empty. No data was found. Check the DATA_PATH and dataset structure.")
    else:
        print(f"\nDataset loaded successfully with {len(dataset_train)} samples.")
        
        # The new dataloader has a map from name to class index. We need the reverse for printing.
        idx_to_name_map = {v: k for k, v in dataset_train.classes_map.items()}
        
        # Get a random item
        random_index = random.randint(0, len(dataset_train) - 1)
        print(f"Fetching random sample at index: {random_index}")
        
        # The __getitem__ method returns a tuple of tensors
        points_tensor, cls_label_tensor, seg_labels_tensor = dataset_train[random_index]   # first is all points and their coordinates second is object category third is label pr points
        
        # --- NEW LOGIC TO GET CATEGORY NAME (continued) ---
        # Get the category index from the tensor, then look up its name
        cat_idx = cls_label_tensor.item()
        cat_name = idx_to_name_map.get(cat_idx, f"UnknownCategory_{cat_idx}")
        
        print(f"\n--- Sample Details for Category: {cat_name} ---")
        
        # Check shapes
        print(f"Points tensor shape:      {points_tensor.shape} (Expected: [{config.N_POINTS}, 3])")
        print(f"Class label tensor shape:   {cls_label_tensor.shape} (Expected: [1])")
        print(f"Seg labels tensor shape:  {seg_labels_tensor.shape} (Expected: [{config.N_POINTS}])")
        
        # Check dtypes
        print(f"\nPoints tensor dtype:      {points_tensor.dtype}")
        print(f"Class label tensor dtype:   {cls_label_tensor.dtype}")
        print(f"Seg labels tensor dtype:  {seg_labels_tensor.dtype}")
        
        # Check content
        class_label = cls_label_tensor.item()
        print(f"\nClass label value: {class_label}")
        
        # This part of the check remains the same and is still the most important one
        unique_labels, counts = np.unique(seg_labels_tensor.numpy(), return_counts=True)
        print("\n--- Segmentation Label Analysis ---")
        print("This is the most crucial check. If you see multiple labels, the loader is working.")
        print(f"Unique part labels found in sample: {unique_labels}")
        print(f"Point counts for each label:      {counts}")

        if len(unique_labels) <= 1:
            print("\n[WARNING] Only one unique label was found. The part segmentation data may not be loading correctly.")
        else:
            print("\n[SUCCESS] Multiple unique labels found. The dataset appears to be loading part data correctly.")


        print(type(dataset_train))

--- Testing ShapeNetPart HDF5 Dataset ---
Loading data from: /kaggle/input/shapenetpart/shapenetpart_hdf5_2048
Subset: train, Points per sample: 2048
Loading H5 files for 'train' split: ['train3.h5', 'train5.h5', 'train1.h5', 'train4.h5', 'train2.h5', 'train0.h5']
The size of train data is 12137
Number of points per sample: 2048

Dataset loaded successfully with 12137 samples.
Fetching random sample at index: 10158

--- Sample Details for Category: Airplane ---
Points tensor shape:      torch.Size([2048, 3]) (Expected: [2048, 3])
Class label tensor shape:   torch.Size([1]) (Expected: [1])
Seg labels tensor shape:  torch.Size([2048]) (Expected: [2048])

Points tensor dtype:      torch.float32
Class label tensor dtype:   torch.int64
Seg labels tensor dtype:  torch.int64

Class label value: 0

--- Segmentation Label Analysis ---
This is the most crucial check. If you see multiple labels, the loader is working.
Unique part labels found in sample: [0 1 2 3]
Point counts for each label:     

In [4]:
class DummyConfig2:
        def __init__(self):
            # --- IMPORTANT ---
            # This path must point to the folder containing the .h5 files
            self.DATA_PATH = '/kaggle/input/shapenetpart/shapenetpart_hdf5_2048'
            self.N_POINTS = 2048
            self.subset = 'test' 



config2 = DummyConfig2()
dataset_test = ShapeNetPartH5(config2)
print(len(dataset_test))



Loading H5 files for 'test' split: ['test1.h5', 'test0.h5']
The size of test data is 2874
Number of points per sample: 2048
2874


In [5]:
category_to_parts = dataset_train.seg_classes
category_to_parts
category_to_index = dataset_train.classes_map
category_to_index
index_to_category = {v:k for k,v in category_to_index.items()}
index_to_category

{0: 'Airplane',
 1: 'Bag',
 2: 'Cap',
 3: 'Car',
 4: 'Chair',
 5: 'Earphone',
 6: 'Guitar',
 7: 'Knife',
 8: 'Lamp',
 9: 'Laptop',
 10: 'Motorbike',
 11: 'Mug',
 12: 'Pistol',
 13: 'Rocket',
 14: 'Skateboard',
 15: 'Table'}

In [6]:
class DummyConfig3:
        def __init__(self):
            # --- IMPORTANT ---
            # This path must point to the folder containing the .h5 files
            self.DATA_PATH = '/kaggle/input/shapenetpart/shapenetpart_hdf5_2048'
            self.N_POINTS = 2048
            self.subset = 'val' 

config3 = DummyConfig3()
dataset_val = ShapeNetPartH5(config3)
print(len(dataset_val))

Loading H5 files for 'val' split: ['val0.h5']
The size of val data is 1870
Number of points per sample: 2048
1870


In [7]:
import plotly.express as px
import pandas as pd
import numpy as np

def visualize_point_cloud_interactive(sample):
    """
    sample: a tuple from your dataset, e.g., (points, labels) or (points, labels, seg)
    """
    points,classes,labels = sample
    # labels = dataset_train[1] if len(sample) > 1 else np.zeros(points.shape[0])
    
    # Convert to NumPy if torch tensor
    # if not isinstance(points, np.ndarray):
    #     points = points.numpy()
    # if not isinstance(labels, np.ndarray):
    #     labels = labels.numpy()
    
    df = pd.DataFrame({
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "label": labels
    })
    
    fig = px.scatter_3d(
        df, x="x", y="y", z="z", color="label",
        labels={"label": "Classes"}, opacity=0.7
    )
    fig.update_traces(marker=dict(size=3, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))
    fig.update_layout(
        title="Interactive Point Cloud Visualization",
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
        legend_title="Labels"
    )
    fig.show()

# Example usage
visualize_point_cloud_interactive(dataset_train[100])
# visualize_point_cloud_interactive(dataset_train[300])


## New implementation

In [83]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_single_segmentation_with_category(points, seg_gt, seg_pred, category_name="Unknown", shape_idx=0):
    """
    points: (N, 3) numpy array of point cloud coordinates
    seg_gt: (N,) numpy array of ground truth labels
    seg_pred: (N,) numpy array of predicted labels
    category_name: string label for the object category
    shape_idx: integer index of the object
    """

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
        subplot_titles=[
            f"Ground Truth - {category_name} (Object {shape_idx})",
            f"Prediction - {category_name} (Object {shape_idx})"
        ]
    )

    # Ground truth segmentation
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=seg_gt,
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title="GT Labels")
        ),
        name="Ground Truth"
    ), row=1, col=1)

    # Predicted segmentation
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=seg_pred,
            colorscale='Rainbow',
            opacity=0.8,
            colorbar=dict(title="Predicted Labels")
        ),
        name="Prediction"
    ), row=1, col=2)

    fig.update_layout(
        title=f"Segmentation Comparison for Category: {category_name}",
        width=1200,
        height=600
    )

    fig.show()


In [60]:
class PointNetMLP(nn.Module):
    def __init__(self, in_channels, mlp_channels):
        super().__init__()
        layers = []
        last_c = in_channels
        for out_c in mlp_channels:
            layers.append(nn.Conv2d(last_c, out_c, 1))
            layers.append(nn.BatchNorm2d(out_c))
            layers.append(nn.ReLU())
            last_c = out_c
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        # print("MLP input:", x.shape)
        return self.mlp(x)


In [61]:
class PointNetSetAbstraction(nn.Module):
    def __init__(self, npoint, radius, nsample, in_channel, mlp):
        super().__init__()
        self.npoint = npoint
        self.radius = radius
        self.nsample = nsample
        self.mlp = PointNetMLP(in_channel + 3, mlp)

    def forward(self, xyz, points):
        # xyz: (B, N, 3), points: (B, C, N) or None
        B, N, _ = xyz.shape
        if self.npoint is None:
            # global SA
            if points is not None:
                grouped = torch.cat([xyz.transpose(1, 2), points], 1)
            else:
                grouped = xyz.transpose(1, 2)
            grouped = grouped.unsqueeze(-1)
            new_points = self.mlp(grouped)
            new_points = torch.max(new_points, 2)[0]
            return None, new_points

        # 1) sample centroids
        new_xyz_idx = farthest_point_sample(xyz, self.npoint)
        new_xyz = index_points(xyz, new_xyz_idx)

        # 2) group neighbors
        group_idx = query_ball_point(self.radius, self.nsample, xyz, new_xyz)
        grouped_xyz = index_points(xyz, group_idx)
        grouped_xyz_norm = grouped_xyz - new_xyz.unsqueeze(2)

        if points is not None:
            grouped_points = index_points(points.transpose(1, 2), group_idx)
            grouped = torch.cat([grouped_xyz_norm, grouped_points], dim=-1)
        else:
            grouped = grouped_xyz_norm

        grouped = grouped.permute(0, 3, 2, 1)
        new_points = self.mlp(grouped)
        new_points = torch.max(new_points, 2)[0]
        # print("SA output -> new_xyz:", new_xyz.shape if new_xyz is not None else None,
        #       "new_points:", new_points.shape)
        return new_xyz, new_points


In [75]:
class PointNetFeaturePropagation(nn.Module):
    def __init__(self, in_channels, mlp_channels):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Conv1d(in_channels, mlp_channels[0], 1),
            nn.BatchNorm1d(mlp_channels[0]),
            nn.ReLU(),
            nn.Conv1d(mlp_channels[0], mlp_channels[1], 1),
            nn.BatchNorm1d(mlp_channels[1]),
            nn.ReLU()
        )

    def forward(self, xyz1, xyz2, points1, points2):
        """
        Interpolate from xyz2->xyz1 (upsampling)
        xyz1: target (denser) points, (B, N1, 3)
        xyz2: source (sparser) points, (B, N2, 3)
        points1: skip features from encoder (B, C1, N1)
        points2: features from decoder (B, C2, N2)
        """
        B, N1, _ = xyz1.shape
        B, N2, _ = xyz2.shape

        if N2 == 1:
            interpolated_points = points2.repeat(1, 1, N1)
        else:
            dists = square_distance(xyz1, xyz2)
            dists, idx = dists.sort(dim=-1)
            dists, idx = dists[:, :, :3], idx[:, :, :3]
            dist_recip = 1.0 / (dists + 1e-8)
            norm = torch.sum(dist_recip, dim=2, keepdim=True)
            weight = dist_recip / norm
            interpolated_points = torch.sum(index_points(points2.transpose(1, 2), idx) * weight.unsqueeze(-1), dim=2).transpose(1, 2)

        if points1 is not None:
            new_points = torch.cat([points1, interpolated_points], dim=1)
        else:
            new_points = torch.cat([xyz1.transpose(1, 2), interpolated_points], dim=1)

        # print("FP input:", new_points.shape)
        return self.mlp(new_points)


In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ------------------------
# Utils: Distance, Indexing, Sampling, Grouping
# ------------------------

def square_distance(src, dst):
    # src: (B, N, 3), dst: (B, M, 3)
    # print(f"src shape is:{src.shape},dst shape is:{dst.shape}")
    dist = torch.sum((src.unsqueeze(2) - dst.unsqueeze(1)) ** 2, -1)
    # print("square_distance ->", dist.shape)  # (B, N, M)
    return dist

def index_points(points, idx):
    """
    Input:
        points: (B, N, C)
        idx: (B, npoint) or (B, npoint, nsample)
    Return:
        new_points: (B, npoint, C) or (B, npoint, nsample, C)
    """
    B, N, C = points.shape
    device = points.device

    # Clamp indices to prevent out-of-bounds access
    idx = torch.clamp(idx, 0, N - 1)

    # Prepare batch index tensor for fancy indexing
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)  # [B,1,1,...]
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1                           # [1,npoint,nsample,...]

    batch_indices = torch.arange(B, dtype=torch.long, device=device).view(view_shape).repeat(repeat_shape)

    new_points = points[batch_indices, idx, :]

    return new_points



def farthest_point_sample(xyz, npoint):
    # xyz: (B, N, 3) -> returns (B, npoint)
    B, N, _ = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long, device=xyz.device)
    distance = torch.ones(B, N, device=xyz.device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long, device=xyz.device)
    batch_indices = torch.arange(B, dtype=torch.long, device=xyz.device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid = xyz[batch_indices, farthest, :].view(B, 1, 3)
        dist = torch.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, -1)[1]
    # print("farthest_point_sample ->", centroids.shape)
    return centroids

def query_ball_point(radius, nsample, xyz, new_xyz):
    # xyz: (B, N, 3), new_xyz: (B, S, 3)
    sqrdists = square_distance(new_xyz, xyz)  # (B, S, N)
    group_idx = torch.argsort(sqrdists, dim=-1)[:, :, :nsample]  # (B, S, nsample)
    # print("query_ball_point ->", group_idx.shape)
    return group_idx


In [76]:
import torch.nn.functional as F

class SegmentationHead(nn.Module):
    def __init__(self, in_channels, num_classes, num_categories, cat_dim=64):
        super().__init__()
        self.cat_fc = nn.Linear(num_categories, cat_dim)
        self.conv1 = nn.Conv1d(in_channels + cat_dim, 128, 1)
        self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = nn.Conv1d(128, num_classes, 1)

    def forward(self, x, category):
        B, C, N = x.shape
        cat_onehot = F.one_hot(category, num_classes=self.cat_fc.in_features).float().to(x.device)
        cat_feat = self.cat_fc(cat_onehot).unsqueeze(2).repeat(1, 1, N)
        x = torch.cat([x, cat_feat], dim=1)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.conv2(x)
        x = x.transpose(2, 1).contiguous()  # shape (B, N, num_classes)
        return F.log_softmax(x, dim=-1)     # log-probabilities for NLLLoss


In [77]:
class PointNet2Seg(nn.Module):
    def __init__(self, num_classes, num_categories):
        super().__init__()
        self.sa1 = PointNetSetAbstraction(512, 0.2, 32, 0, [64, 64, 128])
        self.sa2 = PointNetSetAbstraction(128, 0.4, 64, 128, [128, 128, 256])
        self.sa3 = PointNetSetAbstraction(None, None, None, 256, [256, 512, 1024])

        self.fp3 = PointNetFeaturePropagation(1024 + 256, [256, 256])
        self.fp2 = PointNetFeaturePropagation(256 + 128, [256, 128])
        self.fp1 = PointNetFeaturePropagation(128 + 3, [128, 128])

        self.seg_head = SegmentationHead(128, num_classes, num_categories)

    def forward(self, xyz, category):
        l1_xyz, l1_points = self.sa1(xyz, None)
        l2_xyz, l2_points = self.sa2(l1_xyz, l1_points)
        l3_xyz, l3_points = self.sa3(l2_xyz, l2_points)

        l2_points = self.fp3(l2_xyz, l3_xyz if l3_xyz is not None else l2_xyz, l2_points, l3_points)
        l1_points = self.fp2(l1_xyz, l2_xyz, l1_points, l2_points)
        l0_points = self.fp1(xyz, l1_xyz, None, l1_points)

        out = self.seg_head(l0_points, category)
        # print("Final output:", out.shape)
        return out


In [78]:
if __name__ == "__main__":
    B, N = 2, 1024
    num_classes = 50
    num_categories = 16
    xyz = torch.rand(B, N, 3)
    category = torch.randint(0, num_categories, (B,))

    model = PointNet2Seg(num_classes, num_categories)
    out = model(xyz, category)
    print("✅ Model works, output shape:", out.shape)


✅ Model works, output shape: torch.Size([2, 1024, 50])


In [79]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# ------------------------
# Example: training loop with tqdm
# ------------------------
NUM_CLASSES = 50
NUM_CATEGORIES = len(category_to_index)
EPOCHS = 30
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False)

model = PointNet2Seg(num_classes=NUM_CLASSES, num_categories=NUM_CATEGORIES).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)

criterion = nn.NLLLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss, total_correct, total_points = 0.0, 0, 0

    # tqdm for training batches
    train_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{EPOCHS}] Train", leave=False)
    for points, category, seg in train_bar:
        points = points.to(DEVICE, dtype=torch.float32)            # (B, N, 3)
        category = category.to(DEVICE).squeeze()                   # (B,)
        seg = seg.to(DEVICE, dtype=torch.long)                     # (B, N)

        optimizer.zero_grad()
        # print(f"giving points shape:{points.shape}")
        preds = model(points, category)                     # (B, N, num_classes)
        preds_flat = preds.view(-1, NUM_CLASSES)                   # (B*N, num_classes)
        seg_flat = seg.view(-1)                                    # (B*N,)

        loss = criterion(preds_flat, seg_flat)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred_labels = preds_flat.argmax(1)
        total_correct += pred_labels.eq(seg_flat).sum().item()
        total_points += seg_flat.numel()

        # live tqdm updates
        train_bar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Acc': f"{(total_correct/total_points):.4f}"
        })

    train_loss = total_loss / len(train_loader)
    train_acc = total_correct / total_points

    # validation
    model.eval()
    val_loss, val_correct, val_points = 0.0, 0, 0
    val_bar = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{EPOCHS}] Val", leave=False)
    with torch.no_grad():
        for points, category, seg in val_bar:
            points = points.to(DEVICE, dtype=torch.float32)
            category = category.to(DEVICE).squeeze()
            seg = seg.to(DEVICE, dtype=torch.long)

            preds = model(points, category)
            preds_flat = preds.view(-1, NUM_CLASSES)
            seg_flat = seg.view(-1)

            loss = criterion(preds_flat, seg_flat)
            val_loss += loss.item()
            val_correct += preds_flat.argmax(1).eq(seg_flat).sum().item()
            val_points += seg_flat.numel()

            val_bar.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'Acc': f"{(val_correct/val_points):.4f}"
            })

    val_loss /= len(val_loader)
    val_acc = val_correct / val_points

    print(f"Epoch {epoch+1}/{EPOCHS} | "
          f"Train Loss {train_loss:.4f} Acc {train_acc:.4f} | "
          f"Val Loss {val_loss:.4f} Acc {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_pointnet2seg_model.pth")
        print(f"Best model saved with val_acc: {best_val_acc:.4f}")

Epoch 1/30 | Train Loss 0.8719 Acc 0.7273 | Val Loss 0.4409 Acc 0.8323
Best model saved with val_acc: 0.8323


Epoch 2/30 | Train Loss 0.4185 Acc 0.8435 | Val Loss 0.3478 Acc 0.8702
Best model saved with val_acc: 0.8702


Epoch 3/30 | Train Loss 0.3530 Acc 0.8689 | Val Loss 0.3454 Acc 0.8734
Best model saved with val_acc: 0.8734


Epoch 4/30 | Train Loss 0.3233 Acc 0.8805 | Val Loss 0.2938 Acc 0.8911
Best model saved with val_acc: 0.8911


Epoch 5/30 | Train Loss 0.3070 Acc 0.8858 | Val Loss 0.2882 Acc 0.8954
Best model saved with val_acc: 0.8954


...

Epoch 26/30 | Train Loss 0.2014 Acc 0.9253 | Val Loss 0.2155 Acc 0.9234


Epoch 27/30 | Train Loss 0.2028 Acc 0.9247 | Val Loss 0.2011 Acc 0.9285
Best model saved with val_acc: 0.9285


Epoch 28/30 | Train Loss 0.1983 Acc 0.9264 | Val Loss 0.2030 Acc 0.9272


Epoch 29/30 | Train Loss 0.1964 Acc 0.9268 | Val Loss 0.2005 Acc 0.9285
Best model saved with val_acc: 0.9285


Epoch 30/30 | Train Loss 0.1932 Acc 0.9280 | Val Loss 0.2058 Acc 0.9248


In [80]:
import torch
import torch.nn.functional as F
import time
import numpy as np
import psutil
import os
test_loader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)
def compute_metrics(model, dataloader, num_classes, device):
    model.eval()
    total_correct, total_seen = 0, 0
    total_loss = 0.0
    all_iou_per_instance = []
    part_intersection = np.zeros(num_classes)
    part_union = np.zeros(num_classes)
    criterion = torch.nn.NLLLoss()

    start_time = time.time()
    with torch.no_grad():
        for points, category, seg in dataloader:
            points = points.to(DEVICE, dtype=torch.float32)            # (B, N, 3)
            category = category.to(DEVICE).squeeze()                   # (B,)
            seg = seg.to(DEVICE, dtype=torch.long)                     # (B, N)
    
            optimizer.zero_grad()
            # print(f"giving points shape:{points.shape}")
            preds = model(points, category)                     # (B, N, num_classes)
            preds_flat = preds.view(-1, NUM_CLASSES)                   # (B*N, num_classes)
            seg_flat = seg.view(-1)                                    
            loss = criterion(preds_flat, seg_flat)
        
            total_loss += loss.item()

            pred_choice = preds_flat.argmax(1)
            total_correct += pred_choice.eq(seg_flat).sum().item()
            total_seen += seg_flat.numel()

            # --- Compute IoU per instance ---
            preds_np = pred_choice.cpu().numpy().reshape(points.size(0), -1)
            seg_np = seg.cpu().numpy().reshape(points.size(0), -1)
            for shape_idx in range(points.size(0)):
                part_iou = []
                for part in np.unique(seg_np[shape_idx]):
                    I = np.sum((preds_np[shape_idx] == part) & (seg_np[shape_idx] == part))
                    U = np.sum((preds_np[shape_idx] == part) | (seg_np[shape_idx] == part))
                    if U == 0:
                        iou = 1.0
                    else:
                        iou = I / float(U)
                    part_iou.append(iou)
                    part_intersection[part] += I
                    part_union[part] += U
                all_iou_per_instance.append(np.mean(part_iou))

    # --- Metrics ---
    total_time = time.time() - start_time
    num_samples = len(dataloader.dataset)
    avg_inference_time = total_time / num_samples

    overall_acc = total_correct / total_seen
    instance_miou = np.mean(all_iou_per_instance)
    class_miou = np.mean(part_intersection / np.maximum(part_union, 1e-6))

    # --- Memory ---
    process = psutil.Process(os.getpid())
    memory_mb = process.memory_info().rss / 1024 ** 2  # in MB

    # --- Model size ---
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    model_size_mb = (param_size + buffer_size) / 1024 ** 2

    return {
        'loss': total_loss / len(dataloader),
        'overall_acc': overall_acc,
        'instance_miou': instance_miou,
        'class_miou': class_miou,
        'avg_inference_time': avg_inference_time,
        'memory_mb': memory_mb,
        'model_size_mb': model_size_mb
    }


In [81]:
metrics = compute_metrics(model, test_loader, NUM_CLASSES, DEVICE)

print(f"\nValidation Metrics:")
print(f"Loss:              {metrics['loss']:.4f}")
print(f"Overall Accuracy:  {metrics['overall_acc']:.4f}")
print(f"Instance mIoU:     {metrics['instance_miou']:.4f}")
print(f"Class mIoU:        {metrics['class_miou']:.4f}")
print(f"Inference Time:    {metrics['avg_inference_time']*1000:.2f} ms per sample")
print(f"Memory Usage:      {metrics['memory_mb']:.2f} MB")
print(f"Model Size:        {metrics['model_size_mb']:.2f} MB\n")



Validation Metrics:
Loss:              0.2682
Overall Accuracy:  0.9015
Instance mIoU:     0.7678
Class mIoU:        0.6714
Inference Time:    9.39 ms per sample
Memory Usage:      2840.62 MB
Model Size:        5.37 MB



In [84]:
model.eval()
device = DEVICE
num_classes = NUM_CLASSES

# Keep track of which categories we've already visualized
shown_categories = set()

with torch.no_grad():
    for points, category, seg in test_loader:
        points = points.to(device, dtype=torch.float32)
        category = category.to(device, dtype=torch.long).squeeze()
        seg = seg.to(device, dtype=torch.long)

        preds = model(points, category)
                         # (B*N, num_classes)
                                         
        
    
        

        pred_choice = preds_flat.argmax(1)
        preds_flat = preds.view(-1, num_classes)
        seg_flat = seg.view(-1)

        pred_choice = preds_flat.argmax(1)
        preds_np = pred_choice.cpu().numpy().reshape(points.size(0), -1)
        seg_np = seg.cpu().numpy().reshape(points.size(0), -1)

        # Iterate over batch items
        for b in range(points.size(0)):
            cat_id = category[b].item()
            cat_name = index_to_category[cat_id]

            # Skip if already visualized this category
            if cat_name in shown_categories:
                continue

            # Plot one sample of this category
            plot_single_segmentation_with_category(
                points[b].cpu().numpy(),
                seg_np[b],
                preds_np[b],
                category_name=cat_name,
                shape_idx=b
            )

            shown_categories.add(cat_name)
            print(f"✅ Shown category: {cat_name} ({len(shown_categories)}/16)")

            # Stop once all 16 have been shown
            if len(shown_categories) == 16:
                print("\n🎉 Displayed one sample from each of the 16 categories. Done!")
                break

        if len(shown_categories) == 16:
            break

        


✅ Shown category: Car (1/16)


✅ Shown category: Table (2/16)


✅ Shown category: Lamp (3/16)


✅ Shown category: Guitar (4/16)


✅ Shown category: Chair (5/16)


...

✅ Shown category: Bag (12/16)


✅ Shown category: Earphone (13/16)


✅ Shown category: Skateboard (14/16)


✅ Shown category: Cap (15/16)


✅ Shown category: Rocket (16/16)

🎉 Displayed one sample from each of the 16 categories. Done!
